In [28]:
import pandas as pd
import os
current_dir = os.getcwd()
current_dir
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")
current_dir = os.getcwd()
current_dir
#set current working directory to H:\VScode\March Group\March_Team_Project\
os.chdir("H:\\VScode\\March Group\\March_Team_Project\\")

You set a new current directory


In [29]:
# Issue 16 and Issue 15
# Merge weather data with pollution data, ensure pollution data is filtered for locations and timeperiod.
# Merge on date and location
# Save the merged data to a new csv file
# load pollution data
pollution_data = pd.read_csv('Source_Data\\filtered_pollution.zip')
# count of records in pollution data
print(pollution_data.shape)
# load pollution filter "pollution_data_available.csv
# filter pollution data for locations and timeperiod
pollution_filter = pd.read_csv('Outputs/pollution_data_available.csv')
# count of records in pollution filter
print(pollution_filter.shape)
# filter pollution_data for location in polution_filter with date between years 2003 and 2016
pollution_data = pollution_data[pollution_data['Date Local'].str.contains('200[3-9]|201[0-6]')]
print(pollution_data.shape)
pollution_data = pollution_data[pollution_data['State'].isin(pollution_filter['State'])]
print(pollution_data.shape)
pollution_data

(158438, 30)
(29, 6)
(158438, 30)
(158438, 30)


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,Year
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2003-01-01,Parts per billion,...,4.333333,10.0,0,14.0,Parts per million,2.237500,5.9,0,NaN,2003
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2003-01-01,Parts per billion,...,4.333333,10.0,0,14.0,Parts per million,2.089474,4.6,5,52.0,2003
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2003-01-01,Parts per billion,...,4.300000,8.3,2,NaN,Parts per million,2.237500,5.9,0,NaN,2003
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2003-01-01,Parts per billion,...,4.300000,8.3,2,NaN,Parts per million,2.089474,4.6,5,52.0,2003
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2003-01-02,Parts per billion,...,4.625000,10.0,6,14.0,Parts per million,1.795652,3.4,6,NaN,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158433,127305,48,453,14,3724 North Hills Dr,Texas,Travis,Austin,2013-12-30,Parts per billion,...,0.037500,0.2,23,NaN,Parts per million,0.091667,0.3,21,3.0,2013
158434,127306,48,453,14,3724 North Hills Dr,Texas,Travis,Austin,2013-12-31,Parts per billion,...,2.104167,10.1,14,14.0,Parts per million,0.200000,0.4,0,NaN,2013
158435,127307,48,453,14,3724 North Hills Dr,Texas,Travis,Austin,2013-12-31,Parts per billion,...,2.104167,10.1,14,14.0,Parts per million,0.225000,0.4,0,5.0,2013
158436,127308,48,453,14,3724 North Hills Dr,Texas,Travis,Austin,2013-12-31,Parts per billion,...,2.062500,7.3,14,NaN,Parts per million,0.200000,0.4,0,NaN,2013


In [30]:
# provide a count of records by year
pollution_data['year'] = pollution_data['Date Local'].str[:4]
pollution_data.groupby('year').size()   


year
2003    75936
2013    82502
dtype: int64

In [ ]:
# create a cleaned version of pollution data where any row with a null value is removed.
pollution_data_cleaned = pollution_data.dropna()
print(pollution_data_cleaned.shape)
#format city as capitals
pollution_data_cleaned['City'] = pollution_data_cleaned['City'].str.upper()
# rename Date Local to Date
pollution_data_cleaned.rename(columns={'Date Local':'Date'}, inplace=True)
#convert Date to datetime
pollution_data_cleaned['Date'] = pd.to_datetime(pollution_data_cleaned['Date'])
pollution_data_cleaned

In [ ]:
# load weather data on day and city
weather_data = pd.read_csv('Outputs/Us_Weather_Final_10km_V2.zip')
# rename columns to match stations
weather_data = weather_data.rename(columns={'Station_ID':'StationId'})
# convert date to datetime current format yyyymmdd
weather_data['Date'] = weather_data['Date'].astype(str)
weather_data['Date'] = weather_data['Date'].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
weather_data['Date'] = pd.to_datetime(weather_data['Date'])
# drop columns not needed
weather_data = weather_data.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
print(weather_data.shape)
weather_data

In [ ]:
# load Us_Stations_with_City_10km.csv
stations = pd.read_csv('Outputs/Us_Stations_with_City_10km.csv')
# format StationId as capitals
stations['StationId'] = stations['StationId'].str.upper()
# format city as capitals
stations['City'] = stations['City'].str.upper()
print(stations.shape)
stations

In [ ]:
# merge weather data with stations on station id
weather_data = pd.merge(weather_data, stations, on='StationId')
print(weather_data.shape)
# set all null values to 0
weather_data = weather_data.fillna(0)
# rename columns Latitude and Longitude to WS_Latitude and WS_Longitude
weather_data = weather_data.rename(columns={'Latitude':'WS_Latitude', 'Longitude':'WS_Longitude', "Elevation": "WS_Elevation", "Name": "WS_Name"})

weather_data

In [ ]:
# merge pollution data with weather data on date and location
merged_data = pd.merge(pollution_data_cleaned, weather_data, left_on=['Date', 'City'], right_on=['Date', 'City'], how='inner')
print(merged_data.shape)
merged_data

In [ ]:
# drop columns not needed
merged_data = merged_data.drop(columns=["Unnamed: 0"])

#  display Percentage of null values in each column
null_values_percentage = merged_data.isnull().sum() / len(merged_data) * 100
null_values_percentage


In [ ]:
# Save merged to zip file
merged_data.to_csv('Outputs/Merged_Data_PG_1.zip', index=False)

In [ ]:
# convert "State Code", "County Code", "Site Num" to object
merged_data[["State Code", "County Code", "Site Num"]] = merged_data[["State Code", "County Code", "Site Num"]].astype(str)

# produce list of numeric and bool columns
numeric_columns = merged_data.select_dtypes(include=['float64', 'int64', 'bool']).columns
# drop WS_Latitude, WS_Longitude from numeric_columns
numeric_columns = numeric_columns.drop(['WS_Latitude', 'WS_Longitude'])

print(numeric_columns)
# produce correlation matrix of merged_data for columns in numeric_columns
correlation_matrix = merged_data[numeric_columns].corr()


In [ ]:
# round all values in correlation matrix to 2 decimal places
correlation_matrix = correlation_matrix.round(2)
correlation_matrix

In [ ]:
# Create a visualisation of correlation matrix using seaborn heatmap with negatives being reds and positives being blues
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".1f")

plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Use scikit learn to apply transformations to the data to standardise the data and correct skewness
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# create a list of numeric columns
numeric_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
# drop WS_Latitude, WS_Longitude from numeric_columns
numeric_columns = numeric_columns.drop(['WS_Latitude', 'WS_Longitude'])

# create a list of columns with skewness greater than 1
skewness = merged_data[numeric_columns].skew()
skewness = skewness[skewness > 1]
skewness

# apply power transformer to columns with skewness greater than 1
power_transformer = PowerTransformer()
merged_data[skewness.index] = power_transformer.fit_transform(merged_data[skewness.index])

# apply standard scaler to all numeric columns
scaler = StandardScaler()
merged_data[numeric_columns] = scaler.fit_transform(merged_data[numeric_columns])

# perform correlation matrix again
correlation_matrix2 = merged_data[numeric_columns].corr()
correlation_matrix2 = correlation_matrix2.round(2)
correlation_matrix2

# Create a visualisation of correlation matrix using seaborn heatmap with negatives being reds and positives being blues
plt.figure(figsize=(16, 6))
sns.heatmap(correlation_matrix2, annot=True, cmap='coolwarm', fmt=".1f")
plt.title('Correlation Matrix')
plt.show()







In [ ]:
# minus correlation matrix from correlation matrix2
correlation_matrix_diff = correlation_matrix - correlation_matrix2
correlation_matrix_diff

# Create a visualisation of correlation matrix using seaborn heatmap with negatives being reds and positives being blues
plt.figure(figsize=(16, 6))
sns.heatmap(correlation_matrix_diff, annot=True, cmap='coolwarm', fmt=".1f")
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Compare the correlation matrix before and after the transformations
# create a new column transformer
column_transformer = ColumnTransformer(transformers=[('power_transformer', PowerTransformer(), skewness.index), ('standard_scaler', StandardScaler(), numeric_columns)])

# create a new pipeline
pipeline = Pipeline(steps=[('column_transformer', column_transformer)])

# fit and transform the pipeline
merged_data_transformed = pipeline.fit_transform(merged_data)

# convert transformed data to a dataframe
merged_data_transformed = pd.DataFrame(merged_data_transformed, columns=numeric_columns)
# perform correlation matrix on transformed data

correlation_matrix_transformed = merged_data_transformed.corr()
correlation_matrix_transformed = correlation_matrix_transformed.round(2)
correlation_matrix_transformed



In [ ]:
# preform a linear regression on the data to predict the pollution level
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# create a list of numeric columns
numeric_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
# drop WS_Latitude, WS_Longitude from numeric_columns
numeric_columns = numeric_columns.drop(['WS_Latitude', 'WS_Longitude'])

# split data into features and target
X = merged_data[numeric_columns]
y = merged_data['NO2 AQI']

# split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# predict the target using the model
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
mse

# save the model
import joblib

joblib.dump(model, 'Outputs/Linear_Regression_Model.pkl')

mse



In [ ]:
# load the model
model = joblib.load('Outputs/Linear_Regression_Model.pkl')

# predict the target using the model
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
mse

# create a dataframe of the coefficients
coefficients = pd.DataFrame(model.coef_, index=numeric_columns, columns=['Coefficient'])
coefficients

# create a dataframe of the intercept
intercept = pd.DataFrame([model.intercept_], index=['Intercept'], columns=['Value'])
intercept

# save the coefficients and intercept to csv
coefficients.to_csv('Outputs/Coefficients.csv')
intercept.to_csv('Outputs/Intercept.csv')